In [ ]:
%matplotlib inline
%run ../setup/nb_setup

# Orbits 1: Orbits in Spherical Gravitational Potentials

Author(s): Adrian Price-Whelan, Sarah Pearson


## Learning goals

The main goal for this tutorial is to introduce the concept of orbits and numerically-computed trajectories using spherical gravitational potential models.


## Introduction

For many contexts in Galactic dynamics, the orbital trajectory of an object (star, planet, star cluster, satellite galaxy, etc.) is a fundamental tool for studying and interpreting the dynamical state, evolution, or initial conditions of a gravitational system. However, in general, we cannot directly observe any appreciable changes in position or velocity of a gravitating body over human timescales. As Galactic dynamicists, we therefore effectively only have access to measurements of a single kinematic snapshot of any dynamical system. With a kinematic snapshot and a model for the overall mass distribution of a system, we then seek to compute the past or future trajectories of the constituent bodies and use this information to, for example, refine our mass model, interpret kinematic features, or predict the evolution (past or present) of a system. In many such regimes, the mass of the orbiting body is small relative to the larger mass distribution in which it orbits (e.g., a star within a Galaxy). It is therefore often sufficient to treat an orbiting body as an effectively massless "tracer" particle whose trajectory is set by the broader mass distribution, and which does not cause any appreciable backreaction of the host system. (In later workshops, we will explore cases in which this assumption breaks down, such as with the Milky Way–LMC interaction.) In this context, a common goal is to compute the time-dependent position and velocity — the orbit — of an effectively massless body orbiting within an external gravitational field.

While galaxies are actually composed of a large number of discrete point masses and structures (and we do not know the smallest size of dark matter substructures), it is often sufficient to model the mass distribution of a galaxy by considering smooth mass distributions (where the smoothing length scale is much smaller than the size of a galaxy, <<100 kpc, but larger than the spacing between massive structures, >>1 pc). As we saw in the lectures, and as you might recall from past Mechanics courses, the fundamental quantity in the context of computing orbits is the acceleration field $\boldsymbol{a}(\boldsymbol{x})$ specified often as a function of a three-dimensional position $\boldsymbol{x}$. In convervative systems — or effectively conservative systems: neglecting star formation and death/supernovae for stars, and assuming that dark matter acts effectively like a collisionless fluid, the massive objects that make up a galaxy ineract only through gravitational interactions — the (vector) acceleration field can be directly computed as the gradient of the (scalar) gravitational potential $\Phi(\boldsymbol{x})$ of a system:

$$
\boldsymbol{a}(\boldsymbol{x}) = -\nabla \Phi(\boldsymbol{x})
$$

where the gravitational potential (sometimes later abbreviated simply as "potential") is related to the mass (density) distribution through Poisson's equation:

$$
\nabla^2 \Phi(\boldsymbol{x}) = 4\pi \, G \, \rho(\boldsymbol{x})
$$

While the acceleration or gravitational field is "closer" to the dynamics (in that it is the force on a body that determines how it responds) and the density field is a direct representation of the mass distribution, it is often easier to work with and visualize the gravitational potential. 

In this tutorial, we will introduce some commonly-used spherical gravitational potential models, compute the orbits of particles in some of these models, and analyze the properties of orbits in spherical models.


## Terminology and Notation

- 3D position: $\boldsymbol{x}$
- 3D velocity: $\boldsymbol{v}$
- 6D phase-space position: $\boldsymbol{w} = (\boldsymbol{x}, \boldsymbol{v})$
- Orbit: $\boldsymbol{w}(t) = (\boldsymbol{x}(t), \boldsymbol{v}(t))$
- Gravitational potential: $\Phi(\boldsymbol{x})$
- Mass density: $\rho(\boldsymbol{x})$
- Spherical radius: $r = |\boldsymbol{x}|$
- Unit vector: $\hat{\boldsymbol{x}}$
- Time derivative: $\dot{x} = \textrm{d}x/\textrm{d}t$
- TODO: review whether I should put other terminology here!

### Notebook Setup and Package Imports

In [ ]:
from astropy.constants import G
import astropy.coordinates as coord
import astropy.units as u
from astropy.time import Time

from IPython.display import display, Math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from scipy.signal import argrelmax
import sympy as sy

## Orbits in a Point Mass Potential

One of the simplest spherical mass models is that of a point mass, in which:

$$
\Phi(r) = - \frac{G \, M}{r}
$$

or equivalently

$$
\Phi(\boldsymbol{x}) = - \frac{G \, M}{|\boldsymbol{x}|}
$$

and the acceleration field is

$$
\begin{align}
\boldsymbol{a}(\boldsymbol{x}) &= -\nabla \Phi(\boldsymbol{x}) \\
    & = - \frac{G \, M}{|\boldsymbol{x}|^3} \, \boldsymbol{x} \\
    & = - \frac{G \, M}{r^2} \, \hat{\boldsymbol{x}}
\end{align}
$$

Let's implement a function that computes the point mass acceleration at any 3D position, given a mass M:

In [ ]:
def ptmass_acceleration(xyz, M):
    """
    This function computes the 3D acceleration vector at a 
    position or an array of positions "xyz" from a point mass 
    with mass "M" at the origin.
    
    Parameters
    ----------
    xyz : quantity-like
        This should have shape (3, ...)
    M : quantity-like
        The total mass of the point mass.
    
    """
    r3 = np.sum(xyz**2, axis=0) ** (3/2)
    dPhi_dx = G * M * xyz / r3
    return -dPhi_dx

In the function definition above, note that we use the gravitational constant `G` as imported from `astropy.constants`. This object is more than just a number: It is a special kind of Python/Astropy object called a [`Quantity`](https://docs.astropy.org/en/stable/units/quantity.html#quantity), which carries along and propagates units. For example, if we view the representation of the gravitational constant:

In [ ]:
G

we see that it also displays the units $\frac{\textrm{m}^3}{\textrm{kg} \, \textrm{s}^2}$! When we call the `ptmass_acceleration()` function, this means that we should pass in a position and a point mass with units, and we will get out an acceleration that we can easily convert to other unit representations. For example, assuming the Earth is on the x-axis in some solar system coordinate system, we can evaluate the acceleration felt on Earth by the Sun as:

In [ ]:
earth_pos = [1, 0, 0] * u.au  # 1 Astronomical Unit along the x axis
earth_acc = ptmass_acceleration(earth_pos, M=1 * u.Msun)
earth_acc

This is a strange unit! By default, `astropy.units` propagates units but does not do any conversion or simplification of the units. For example, here the $\textrm{M}_\odot$ in the numerator should cancel the kg in the denominator. If we want to convert this into something more readable, we use the `.to()` method and specify the desired output unit:

In [ ]:
earth_acc.to(u.km / u.s**2)

Some other examples of acceleration units:

In [ ]:
earth_acc.to(u.au / u.yr**2)

In [ ]:
earth_acc.to(u.km / u.s / u.day)

### Example: Compute the acceleration from the Sun at the current (right now!) location of Mars

Get a representation of the current time using `astropy.time.Time()`:

In [ ]:
now = Time.now()
now.mjd

We will use Astropy (`astropy.coordinates`) to retrieve the position of Mars: In the background, this queries the JPL ephemeris for us. So we can retrieve the position of Mars at the current time using:

In [ ]:
mars_pos = coord.get_body('mars', now)
mars_pos

(this works for other solar system bodies - try it! Swap 'mars' out for, e.g., 'saturn')

By default, the position of Mars is returned to us in a Geocentric (Earth-centric) coordinate system. We need to convert this to a Heliocentric coordinate system so we can compute the acceleration from the Sun (assuming it is at the origin). We will convert it to Heliocentric Ecliptic coordinates:

In [ ]:
mars_ecl = mars_pos.transform_to(coord.HeliocentricTrueEcliptic(obstime=now))

Now we can retrieve a Cartesian representation of the Ecliptic position of Mars, and use this to compute the acceleration:

In [ ]:
mars_xyz_now = mars_ecl.cartesian.xyz
mars_xyz_now

In [ ]:
mars_acc = ptmass_acceleration(mars_xyz_now, M=1*u.Msun)
mars_acc.to(u.km / u.s / u.day)

### Exercise: Compute the acceleration on the Moon from the Sun and Earth - which is larger?

*Hint: The Astropy unit `u.Mearth` might be useful!*

*Solution:*

In [ ]:
earth_pos = coord.get_body('earth', now)
moon_pos = coord.get_body('moon', now)

In [ ]:
moon_ecl = moon_pos.transform_to(coord.HeliocentricTrueEcliptic(obstime=now))
moon_xyz_now = moon_ecl.cartesian.xyz
moon_acc_sun = ptmass_acceleration(moon_xyz_now, M=1 * u.Msun)

In [ ]:
moon_acc_earth = ptmass_acceleration(
    (moon_pos.cartesian - earth_pos.cartesian).xyz, 
    M=1 * u.Mearth)

In [ ]:
(np.linalg.norm(moon_acc_sun).to(u.km / u.s / u.day),
 np.linalg.norm(moon_acc_earth).to(u.km / u.s / u.day))

Answer: The acceleration on the Moon from the Sun is larger!

---

### Computing an orbit in the point mass potential

The orbit of a massless particle in any acceleration field can be computed by solving the differential equation

$$
\begin{align}
\ddot{\boldsymbol{x}} &= \boldsymbol{a}(\boldsymbol{x}) \\
\ddot{\boldsymbol{x}} &= -\nabla \Phi(\boldsymbol{x})
\end{align}
$$

For a point mass, this is sometimes referred to as the "Kepler problem": This is a special case in which we could write down an analytic expression for the position of the particle at any time, given a set of initial conditions. However, generically, this differential equation cannot be solved analytically and we must instead resort to numerical integration tools. In this context, it is often easier to reduce the 2nd-order differential equation for postion expressed above into a pair of coupled 1st-order differential equations for position and velocity (or momentum). We do this be introducing an auxiliary variable to represent the 3D velocity, so that the relevant system of differential equations we need to solve is now:

$$
\begin{align}
\dot{\boldsymbol{x}} &= \boldsymbol{v} \\
\dot{\boldsymbol{v}} &= -\nabla \Phi(\boldsymbol{x}) 
\end{align}
$$

We can compress this into a single expression by defining the "phase-space position" $\boldsymbol{w} = (\boldsymbol{x}, \boldsymbol{v})$ such that:

$$
\begin{align}
\dot{\boldsymbol{w}} &= (\dot{\boldsymbol{x}}, \dot{\boldsymbol{v}}) \\
&= (\boldsymbol{v}, -\nabla \Phi(\boldsymbol{x}))
\end{align}
$$

Our goal then is to compute $\boldsymbol{w}(t)$ given initial conditions $\boldsymbol{w}_0$. There are many standard tools available for solving problems like this numerically. This is sometimes called an "initial value problem," or "ordinary differential equation" (ODE) integration. Here, we will define a function to compute the time-derivative of $\boldsymbol{w}$ and use `scipy.integrate.solve_ivp()` to compute the orbital trajectory of a particle ([solve_ivp() documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) for context). 

The function we defined above to compute the point mass acceleration only does half of what we need: We need to define a new function `ptmass_F()` that takes the full phase-space position $\boldsymbol{w}$ and returns its time-derivative, $\dot{\boldsymbol{w}} = (\boldsymbol{v}, -\nabla \Phi(\boldsymbol{x}))$. `solve_ivp()` also requires that the function accept a first argument, the time `t`, which in this case we will include in the definition but ignore in any expressions.

Also, `solve_ivp()` unfortunately does not support `astropy.units`, so we will have to re-implement the point mass acceleration in a way that takes only regular Python numbers. To avoid having to specify units for both `G` and the mass `M`, we will have this function accept the combined parameter `GM = G * M`:

In [ ]:
def ptmass_F(t, w, GM):
    """
    Parameters
    ----------
    t : numeric
        The current time during the integration.
    w : array-like
        The phase-space position, which should have 6 elements.
    GM : numeric
        Mass times the gravitational constant.
    
    """
    xyz = w[:3]
    vxyz = w[3:]
    
    r3 = np.sum(xyz**2, axis=0) ** (3/2)
    dPhi_dx = GM * xyz / r3
    return np.concatenate([vxyz, -dPhi_dx])

Even though we cannot use `astropy.units` to keep track of units internally, we can still use it to do unit conversions before we pass inputs to the function, or with the resulting orbital time series that we compute. For this, it helps to work in a consistent unit system (a set of base units that represent length, mass, and time). Here, since we are using Solar System examples, a good unit system is AU, $\textrm{M}_\odot$, and years:

In [ ]:
unitsys = [u.au, u.Msun, u.year]

### Circular orbits and the circular velocity curve

We now have all of the pieces we need to compute an orbit in the point mass potential! How then should we choose initial conditions for a trial orbit? One simple option is to pick a location, and start a particle on a circular orbit that goes through that location. 

An important property of spherical mass distributions is the concept of the *circular velocity* $v_{\rm circ}(r)$, which is defined to be the speed of a test particle on a circular orbit at radius $r$. The circular velocity can be found by equating the centripetal acceleration $\frac{v^2}{r}$ with the (absolute value of the) acceleration from the gravitational potential $\nabla \Phi$

$$
\begin{align}
\frac{v^2}{r} &= \left|\frac{\textrm{d}\Phi}{\textrm{d}r}\right| \\
v_{\rm circ}(r) &= \sqrt{r \, \left|\frac{\textrm{d}\Phi}{\textrm{d}r}\right|}
\end{align}
$$

For the point mass potential, this is simply:

$$
v_{\rm circ}(r) = \sqrt{\frac{G\,M}{r}}
$$

### Example: Compute the circular velocity in at $r=1.6~\textrm{AU}$ in the Solar System in AU/year

In [ ]:
r0 = 1.6 * u.au
M = 1 * u.Msun
v_circ = np.sqrt(G * M / r0).decompose(unitsys)
v_circ

The `.decompose()` method allows us to represent a `Quantity` in a given unit system without having to construct the unit explicitly (here we just had to pass in the list of base units to `decompose()`).

We will now use this radius and circular velocity to construct 3D initial conditions that we will pass to `solve_ivp()` to compute the orbit of a test particle in the Solar System. We will start the particle on the x-axis ($y=z=0$) so that the velocity (perpendicular to the position vector) is purely in the $\hat{y}$ direction:

In [ ]:
x0 = [1, 0, 0] * r0
v0 = [0, 1, 0] * v_circ
w0 = np.concatenate(
    (x0.decompose(unitsys).value, 
     v0.decompose(unitsys).value)
)
w0

Finally, we have to convert the combined amplitude parameter `GM` into our unit system:

In [ ]:
GM = (G * M).decompose(unitsys).value
GM

The last thing we need to specify is the grid of times over which to compute the orbit. For this example, we will use 1024 equally spaced points between 0 and 10 years, the time unit in our unit system.

In [ ]:
t_eval = np.linspace(0, 10, 1024)

In [ ]:
soln = solve_ivp(
    ptmass_F, 
    y0=w0, 
    args=(GM, ),
    t_span=(t_eval.min(), t_eval.max()), 
    t_eval=t_eval
)

The returned Solution object `soln` now contains the orbit of our test particle (position and velocity) evaluated at all 1024 points in our time grid!

In [ ]:
soln

The Scipy integrators use the variable "`y`" for any integration variables, so in this case `w = y`. To plot the position time series of the particle, we will plot two 2D projections of the 3D position: x-y and x-z:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), 
                         sharex=True, sharey=True,
                         constrained_layout=True)

# x-y
axes[0].plot(soln.y[0], soln.y[1])
axes[0].set_xlabel(f'$x$ [{unitsys[0]:latex_inline}]')
axes[0].set_ylabel(f'$y$ [{unitsys[0]:latex_inline}]')

# x-z
axes[1].plot(soln.y[0], soln.y[2])
axes[1].set_xlabel(f'$x$ [{unitsys[0]:latex_inline}]')
axes[1].set_ylabel(f'$z$ [{unitsys[0]:latex_inline}]')

for ax in axes:
    ax.set_aspect('equal')
    
fig.suptitle('default integrator (RK45)', fontsize=24)

The orbit is confined to the x-y plane, as we might expect (we only gave the particle velocity in the y direction, starting from the x axis). However: **that doesn't look like a circular orbit**: The orbit in the x-y plane looks more like a spiral!?! This is a telltale sign of energy drift. The force on our test particle is conservative and time-independent, so the orbital energy should be conserved along the particle's orbit. Let's compute the total energy for this numerically-integrated orbit and see if it is constant. To do that, we will implement a function to evaluate the point mass potential energy (defined above), and the total energy (the sum of potential and kinetic energy) at any phase-space position:

In [ ]:
def ptmass_potential(xyz, M):
    """
    Compute the potential energy from a point mass at the given
    position(s) ``xyz``.
    
    Parameters
    ----------
    xyz : quantity-like
    M : quantity-like
    """
    r = np.sqrt(np.sum(xyz**2, axis=0))
    return - G * M / r


def ptmass_total_energy(w, M, unitsys):
    """
    Compute the total energy (kinetic + potential) for phase-space 
    position(s) ``w``.
    
    Parameters
    ----------
    w : array-like
    M : quantity-like
    unitsys : list, collection
        A list or iterable containing the base units that define the
        adopted unit system. Order: length, mass, time
    """
    xyz = w[:3] * unitsys[0]
    vxyz = w[3:] * unitsys[0] / unitsys[2]
    E_kin = 0.5 * np.sum(vxyz ** 2, axis=0)
    E_pot = ptmass_potential(xyz, M)
    return E_kin + E_pot

We can now compute the energy for each point in the orbital time series, and the fractional energy change relative to the initial energy `dE`:

In [ ]:
E = ptmass_total_energy(soln.y, M, unitsys)
dE = np.abs((E[1:] - E[0]) / E[0])

plt.figure(figsize=(6, 5))
plt.loglog(dE)
plt.xlabel('timestep number, $j$')
plt.ylabel('fractional energy drift\n' + r'$\left|(E_j - E_0) / E_0\right|$')
plt.ylim(1e-15, 1e0)
plt.suptitle('default integrator (RK45)', fontsize=24)

Aha! The energy of the computed orbit drifts steadily toward a nearly 100% change from the initial energy: This explains why the computed orbit does not look like a circular orbit, as we would expect. The fact that energy is not conserved is a consequence of the default integration method used by `solve_ivp()`: An (effectively) 4th-order Runge–Kutta method. Runge–Kutta methods are class of iterative schemes for computing approximate solutions for ordinary differential equations. However, they are approximate, and so these methods steadily accumulate errors. With `solve_ivp()`, we can control the error tolerance (the internal timestep used to perform the integration) using the function arguments `rtol` and `atol`, which set the threshold relative (rtol) and absolute (atol) error tolerances (see the [solve_ivp() documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) for more context).

Let's try re-running the orbit calculation, but with much smaller `rtol` and `atol` values (compared to the defaults of 1e-3 and 1e-6, respectively):

In [ ]:
soln_tol = solve_ivp(
    ptmass_F, 
    y0=w0, 
    args=(GM, ),
    t_span=(t_eval.min(), t_eval.max()), 
    t_eval=t_eval,
    rtol=1e-12,
    atol=1e-8
)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), 
                         sharex=True, sharey=True,
                         constrained_layout=True)

# x-y
axes[0].plot(soln_tol.y[0], soln_tol.y[1])
axes[0].set_xlabel(f'$x$ [{unitsys[0]:latex_inline}]')
axes[0].set_ylabel(f'$y$ [{unitsys[0]:latex_inline}]')

# x-z
axes[1].plot(soln_tol.y[0], soln_tol.y[2])
axes[1].set_xlabel(f'$x$ [{unitsys[0]:latex_inline}]')
axes[1].set_ylabel(f'$z$ [{unitsys[0]:latex_inline}]')

for ax in axes:
    ax.set_aspect('equal')
    
fig.suptitle('RK45 (improved tolerance)', fontsize=24)

That looks much better! The orbit looks circular, and it visually appears to stay circular and planar for the duration of our integration window. 

We therefore expect the energy conservation to be preserved much more accurately for this case:

In [ ]:
E = ptmass_total_energy(soln_tol.y, M, unitsys)
dE = np.abs((E[1:] - E[0]) / E[0])

plt.figure(figsize=(6, 5))
plt.loglog(dE)
plt.xlabel('timestep number, $j$')
plt.ylabel('fractional energy drift\n' + r'$\left|(E_j - E_0) / E_0\right|$')
plt.ylim(1e-15, 1e0)
plt.title('RK45 (improved tolerance)', fontsize=24)

Indeed, by the end of our integration window, the energy changes by $<10^{-7}$ relative to the initial energy for this case (the axis limits are the same here as in the previous plot! Scroll up to compare the two plots).

## Other Integration Algorithms

Runge–Kutta methods represent one class of numerical ODE integration tools, but there are several other classes of algorithms that are commonly used. Another class of algorithms that is used especially for evolving dynamical systems are *symplectic* integration methods. As we saw above with the Runge–Kutta scheme, over long times, errors introduced from approximations inherent to the method accumulate and lead to an increasing net change in the energy (which should be conserved). Symplectic integrators are special in that the iterative steps in the algorithm are *canonical transformations* themselves. A result of this is that, in conservative systems, over long times or averaged over time, these integrator classes conserve energy (in an average sense) and phase-space volume. In detail, the energy actually oscillates around the true energy, but it does not diverge from approximations in the method and only drifts due to numerical precision, which happens over much longer (integration) timescales.

One of the most well-known symplectic integration methods is the [*Leapfrog algorithm*](https://en.wikipedia.org/wiki/Leapfrog_integration), which is a specific type of more general Verlet integration schemes. Leapfrog is a 2nd-order method (the truncation error is 2nd-order) meaning that at fixed timestep, it will not be as precise as, for example, a 5th-order Runge–Kutta scheme. However, the approximate stability of the energy and the conserved phase volume is often desirable for long integrations. There are also 3rd, 4th, and higher-order analogs to Leapfrog integration that have better error properties (at fixed timestep) over short timescales.


### Leapfrog Integration

The default algorithm for Leapfrog integration advances the position and velocity (momentum) coordinates separately, and out of sync by 1/2 of a timestep, hence the "leapfrog" in the name: When the velocity advances by a full step from (timestep $i - \frac{1}{2}$) to (timestep $i + \frac{1}{2}$), it "leapfrogs" over the position (at timestep $i$). In a bit more detail, Leapfrog works as follows, starting from initial conditions $\boldsymbol{x}_0$ and $\boldsymbol{v}_0$ at time $t=t_0$:

- Compute the acceleration on the particle at the initial time:

  $$
  \boldsymbol{a}(t_0) = -\nabla \Phi|_{\boldsymbol{x}_0}
  $$
  
- Advance the velocity by 1/2 of a timestep $h=\Delta t$:

  $$
  \boldsymbol{v}_{1/2} = \boldsymbol{v}_0 + \frac{h}{2}\,\boldsymbol{a}(t_0)
  $$
  
- For each timestep $i$: 

    - Increment the position by a full timestep using the "old" velocity, $\boldsymbol{v}_{i - 1/2}$
    
      $$
      \boldsymbol{x}_{i} = \boldsymbol{x}_{i-1} + h\,\boldsymbol{v}_{i - 1/2}
      $$
    
    - Compute the acceleration at the "new" position:
    
      $$
      \boldsymbol{a}(t_i) = -\nabla \Phi|_{\boldsymbol{x}_i}
      $$
      
    - Increment the velocity by a full timestep using the "new" acceleration
    
      $$
      \boldsymbol{v}_{i+1/2} = \boldsymbol{v}_{i-1/2} + h\,\boldsymbol{a}(t_i)
      $$
      
      
One caveat is that, if we stored all of the positions $\boldsymbol{x}_{i}$ and velocities $\boldsymbol{v}_{i+1/2}$, they would be out of sync with one another. In practice, it is common to instead split the velocity timestep increment into two increments by half of a timestep, so the last step in each iteration would look like:

- Increment the velocity by two half timesteps using the "new" acceleration
  $$
  \boldsymbol{v}_{i} = \boldsymbol{v}_{i-1/2} + \frac{h}{2}\,\boldsymbol{a}(t_i)
  \boldsymbol{v}_{i+1/2} = \boldsymbol{v}_{i} + \frac{h}{2}\,\boldsymbol{a}(t_i)
  $$
  
where we then save the (synchronized) $\boldsymbol{x}_{i}$ and $\boldsymbol{v}_{i}$ values as the orbital time series.

Let's implement this algorithm as a function, like `solve_ivp()`, which itself takes a function that computes the acceleration as the argument `acc_func`, along with initial conditions `x0` and `v0`, a timestep `dt`, and a number of timesteps `n_steps`. Because we are implementing this ourselves, we can build in support for Astropy units:

In [ ]:
def leapfrog_integrate(acc_func, x0, v0, dt, n_steps, acc_func_args=()):
    """
    Parameters
    ----------
    acc_func : callable
        A function that computes the acceleration at a position.
    x0 : quantity-like [length]
        Initial position.
    v0 : quantity-like [velocity]
        Initial velocity.
    dt : quantity-like [time]
        Timestep.
    n_steps : int
        The number of timesteps to take.
    acc_func_args : iterable, list (optional)
        Any additional arguments to be passed to the `acc_func` function.
    """
    # Make sure the input initial conditions and timestep are valid:
    x0 = u.Quantity(x0)
    v0 = u.Quantity(v0)
    ts = u.Quantity(dt) * np.arange(int(n_steps) + 1)
    
    # Initialize the velocity: step forward by 1/2 timestep
    a0 = acc_func(x0, *acc_func_args)
    v_1_2 = v0 + a0 * dt / 2.
    
    # Set up array Quantity objects to store the full orbits for both 
    # position and velocity
    xs = np.full((3, len(ts)), np.nan) * x0.unit
    xs[:, 0] = x0
    vs = np.full((3, len(ts)), np.nan) * v0.unit
    vs[:, 0] = v0
    
    x_im1 = x0
    v_im1_2 = v_1_2
    for i in range(n_steps):
        x_i = x_im1 + v_im1_2 * dt
        a_i = acc_func(x_i, *acc_func_args)

        v_i = v_im1_2 + a_i * dt / 2
        v_ip1_2 = v_i + a_i * dt / 2
        
        xs[:, i + 1] = x_i
        vs[:, i + 1] = v_i
        
        x_im1 = x_i 
        v_im1_2 = v_ip1_2
        
    return ts, xs, vs

We can now compute the same orbit, using the same point mass potential and initial conditions as above. We will compute it for many more timesteps to compare the long-term behavior of Leapfrog with the Runge–Kutta schemes used above (this takes about 15 seconds on my machine - it is much slower because operations are slower with astropy Quantities as compared to numpy arrays - ask me about how we could speed this up if you are curious!):

In [ ]:
lf_ts, lf_xs, lf_vs = leapfrog_integrate(
    ptmass_acceleration, 
    x0, 
    v0,
    dt=1 * u.day,
    n_steps=16384,
    acc_func_args=(M, )
)

Let's now plot the same projections of the positional components of the orbit to compare with the plots above:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), 
                         sharex=True, sharey=True,
                         constrained_layout=True)

# x-y
axes[0].plot(lf_xs[0], lf_xs[1])
axes[0].set_xlabel(f'$x$ [{unitsys[0]:latex_inline}]')
axes[0].set_ylabel(f'$y$ [{unitsys[0]:latex_inline}]')

# x-z
axes[1].plot(lf_xs[0], lf_xs[2])
axes[1].set_xlabel(f'$x$ [{unitsys[0]:latex_inline}]')
axes[1].set_ylabel(f'$z$ [{unitsys[0]:latex_inline}]')

for ax in axes:
    ax.set_aspect('equal')
    
fig.suptitle('Leapfrog', fontsize=24)

This looks like the same circular orbit that we computed above! How well does it conserve energy?

In [ ]:
E = ptmass_potential(lf_xs, M) + 0.5 * np.sum(lf_vs**2, axis=0)
E = E.decompose(unitsys)
dE = np.abs((E[1:] - E[0]) / E[0])

plt.figure(figsize=(6, 5))
plt.loglog(dE)
plt.xlabel('timestep number, $j$')
plt.ylabel('fractional energy drift\n' + r'$\left|(E_j - E_0) / E_0\right|$')
plt.ylim(1e-15, 1e0)
plt.title('Leapfrog', fontsize=24)

Interestingly, as we expect, the error conservation is different: Rather than steadily rising toward larger numbers of timesteps (compare to the two equivalent figures above), the energy here is not constant, but appears to oscillate around a constant value. We can see this clearer by plotting the energy difference from the mean on a linear-linear plot:

In [ ]:
plt.figure(figsize=(6, 5))
mean_E = np.mean(E)
plt.plot((E - mean_E) / mean_E)
plt.xlabel('timestep number, $j$')
plt.ylabel('energy relative to mean\n' + r'$E_j - {\rm mean}_j(E_j)$')
plt.title('Leapfrog', fontsize=24)
plt.xlim(0, 10000)

The oscillations of the energy are small (on the order of $10^{-10}$ fractional changes in the energy), and appear more or less stable over the timescale that we integrated this orbit over.

We hope that this comparison of Runge–Kutta and Leapfrog methods for the point mass potential gives you a sense that there are often many tools for the job of numerical orbit computation. But which integration scheme should you use? In many contexts in Galactic dynamics, it doesn't make much of a difference: As long as your timestep is small enough that you conserve energy with enough precision for the problem at hand, you can use any of these integration schemes. However, for some problems it may be important to more precisely conserve phase-volume, in which case symplectic integrators are often a safer choice.

In practice, while it is fine to build your own machinery to do orbit integration, there are several tools available for Galactic dynamics that do a lot of the heavy lifting for you (and do it fast!). For example, in Python, your options are:

- [Gala](http://gala.adrian.pw/)
- [Galpy](https://github.com/jobovy/galpy)
- [Agama](http://agama.software/)

most of which implement the integration methods mentioned here (and others!), along with many of the gravitational potential models that we will discuss in the next few sections and tutorials.

### Exercise: Varying the initial conditions

So far, we have only worked with initial conditions for a circular orbit. Compute a few more orbits using `leapfrog_integrate()`, but with different choices for initial conditions (e.g., try different y velocity values, or try generating random velocity vectors). What do generic orbits look like?

## The Plummer Potential

The Kepler potential is special because it represents a point mass, i.e. the density distribution is infinite at its origin and zero everywhere else. Most mass distributions that are relevant for Galactic dynamics are well-represented by density distributions that are finite everywhere or throughout some enclosed volume.

Mathematically, one of the simplest spherical extended (non-point) mass distributions is the [Plummer Sphere](https://en.wikipedia.org/wiki/Plummer_model), given by the potential:

$$
\Phi(r) = - \frac{G \, M}{\sqrt{r^2 + b^2}}
$$

### Exercise: What is the Plummer density distribution?

Using Poisson's equation, the density is related to the Laplacian of the potential such that:

$$
\rho(r) = (4\pi\,G)^{-1} \, \nabla^2 \Phi
$$

Using the spherical expression of the potential, we must take the spherical Laplacian (but the $\partial \phi$ and $\partial \theta$ terms are zero):

$$
\begin{align}
\nabla^2 \Phi(r) &= \frac{1}{r^2} \, \frac{\partial \Phi}{\partial r}\left( r^2 \, \frac{\partial \Phi}{\partial r} \right)\\
&= \frac{1}{r^2} \, \frac{\partial \Phi}{\partial r}\left( \frac{G\,M\,r^3}{(r^2 + b^2)^{3/2}} \right)\\
\rho(r) &= \frac{M}{\frac{4}{3}\pi\,b^3} \, \left(\frac{r^2}{b^2} + 1\right)^{-5/2}
\end{align}
$$

Proof with `sympy`:

In [ ]:
_G, _M, _b, _r = sy.symbols('G, M, b, r', real=True, positive=True)

In [ ]:
plummer_Phi = - _G * _M / sy.sqrt(_r**2 + _b**2)
plummer_Phi

In [ ]:
plummer_d2_Phi = sy.simplify(
    1/_r**2 * sy.diff(_r**2 * sy.diff(plummer_Phi, _r), _r))

In [ ]:
plummer_dens = plummer_d2_Phi / (4*sy.pi * _G)
plummer_dens

### Mass enclosed: Circular velocity in extended mass distributions?

Above, we saw how to compute the circular velocity of an orbit at radius $r$ in the point mass potential. This concept works in any spherical, central potential model, even when the mass is extended, such as in the case of the Plummer model. In more general spherical mass models, the equation for the circular velocity in terms of a derivative of the potential is still valid:

$$
v_{\rm circ}(r) = \sqrt{r \, \left|\frac{\textrm{d}\Phi}{\textrm{d}r}\right|}
$$

However, an alternate way of computing the circular velocity is to use the *mass enclosed* $M(<r)$, which specifies the mass enclosed within a radius $r$ and is defined as:

$$
M(<r) = \int^r_0 \textrm{d}\tilde{r} \, 4\pi\,\tilde{r}^2 \, \rho(\tilde{r})
$$

With the mass enclosed, the circular velocity can also be computed as:

$$
v_{\rm circ}(r) = \sqrt{\frac{G\,M(<r)}{r}}
$$

### Exercise: Implement the Plummer Potential and Compute a Circular Orbit

**First, derive an expression for the mass enclosed in the Plummer model**

*Solution:*

You can either take the derivative of the potential and compute the mass enclosed using the radial derivative of the potential, or compute the integral of the mass density given above. Here, we will use `sympy` to do both. First, using the derivative:

In [ ]:
sy.simplify(sy.sqrt(_r * sy.diff(plummer_Phi, _r)))

In [ ]:
plummer_Menc = sy.integrate(4*sy.pi * _r**2 * plummer_dens, _r)
sy.simplify(sy.sqrt(_G * plummer_Menc / _r))

**Next, implement a function to evaluate the circular velocity in the Plummer potential**

*Solution:*

In [ ]:
def plummer_vcirc(r, M, b):
    return np.sqrt(G * M) * r / (r**2 + b**2)**(3/4)

**Now implement an acceleration function for the Plummer model:**

*Solution:* 

First we have to work out the Cartesian gradient of the potential. For the $x$ term, this is equivalent to:

$$
\begin{align}
\frac{\partial\Phi}{\partial r} &= \frac{G\,M\,r}{(r^2 + b^2)^{3/2}}\\
\frac{\partial\Phi}{\partial x} &= \frac{\partial\Phi}{\partial r} \, \frac{\partial r}{\partial x}\\
&= \frac{\partial\Phi}{\partial r} \, \frac{x}{r}\\
&= \frac{G\,M\,x}{(r^2 + b^2)^{3/2}}
\end{align}
$$

so the $y$ and $z$ terms will be the same, with $x$ swapped out for each respective coordinate value.

In [ ]:
def plummer_acceleration(xyz, M, b):
    """    
    Parameters
    ----------
    xyz : quantity-like
        This should have shape (3, ...)
    M : quantity-like
        The total mass.
    b : quantity-like
        The scale radius.
    
    """
    r2 = np.sum(xyz**2, axis=0)
    dPhi_dr = G * M / (r2 + b**2) ** 1.5
    return - dPhi_dr * xyz

**Finally, compute a circular orbit starting at $(x, y, z) = (5, 0, 0)~\textrm{pc}$ (i.e. $r_0 = 5~\textrm{pc}$) using `leapfrog_integrate()` with the Plummer acceleration function you implemented, adopting the parameter values** $M=10^7~\textrm{M}_\odot$ and $b = 8~\textrm{pc}$

Think about what a good order of magnitude would be for the timestep (*Hint: what is $2\pi\,r_0 / v_\textrm{circ}$?*\)

In [ ]:
r0 = 5 * u.pc
x0 = [1, 0, 0] * r0

plummer_M = 1e7 * u.Msun
plummer_b = 8 * u.pc
vcirc = plummer_vcirc(r0, plummer_M, plummer_b)
print(vcirc.to(u.km/u.s))

v0 = [0, 1, 0] * vcirc

In [ ]:
dt = 2*np.pi*r0 / vcirc / 256  # 256 steps per orbital period
dt.to(u.Myr)

In [ ]:
plummer_ts, plummer_xs, plummer_vs = leapfrog_integrate(
    plummer_acceleration, 
    x0=x0,
    v0=v0,
    dt=dt,
    n_steps=1024,
    acc_func_args=(plummer_M, plummer_b)
)

**Plot the orbit in x-y and x-z projections:**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), 
                         sharex=True, sharey=True,
                         constrained_layout=True)

# x-y
axes[0].plot(plummer_xs[0], plummer_xs[1])
axes[0].set_xlabel(f'$x$ [{plummer_xs.unit:latex_inline}]')
axes[0].set_ylabel(f'$y$ [{plummer_xs.unit:latex_inline}]')

# x-z
axes[1].plot(plummer_xs[0], plummer_xs[2])
axes[1].set_xlabel(f'$x$ [{plummer_xs.unit:latex_inline}]')
axes[1].set_ylabel(f'$z$ [{plummer_xs.unit:latex_inline}]')

for ax in axes:
    ax.set_aspect('equal')
    
fig.suptitle('Circular orbit in the Plummer model', fontsize=24)

**Compute a few other random bound orbits in the Plummer model (using small, random velocity offsets from the circular orbit initial conditions). How are these orbits different from the orbits in the point mass potential?**

In [ ]:
rng = np.random.default_rng(seed=42)

fig, axes = plt.subplots(1, 2, figsize=(10, 5), 
                         sharex=True, sharey=True,
                         constrained_layout=True)

axes[0].set_xlabel(f'$x$ [{plummer_xs.unit:latex_inline}]')
axes[0].set_ylabel(f'$y$ [{plummer_xs.unit:latex_inline}]')

axes[1].set_xlabel(f'$x$ [{plummer_xs.unit:latex_inline}]')
axes[1].set_ylabel(f'$z$ [{plummer_xs.unit:latex_inline}]')

for n in range(4):    
    this_plummer_ts, this_plummer_xs, this_plummer_vs = leapfrog_integrate(
        plummer_acceleration, 
        x0=x0,
        v0=v0 + rng.normal(0, 10, size=3) * u.km/u.s,
        dt=dt,
        n_steps=1024,
        acc_func_args=(plummer_M, plummer_b)
    )
    
    axes[0].plot(this_plummer_xs[0], this_plummer_xs[1], marker='')
    axes[1].plot(this_plummer_xs[0], this_plummer_xs[2], marker='')

Generic orbits in the Plummer model (and any extended spherical mass distribution) are unlike orbits in the point mass potential: In the point mass model, orbits form closed (Keplerian) ellipses, whereas in generic spherical mass models, orbits are *not* closed and instead trace our "rosette" patterns. One property that all orbits in spherical mass models share (including orbits in the point mass model, Plummer model, and any other) is that they are confined to a plane. That plane can be oriented with any arbitrary angles with respect to the adopted coordinate system, but a fundamental property of orbits in spherical mass models is that the motion can be reduced to 2D planar motion with a suitable choice of coordinate system. This is *not* true in more complex mass models (e.g., axisymmetric/disky, triaxial models, or more realistic models of dark matter distributions in galaxies); so why is this the case in spherical models?

## Constants and Integrals of Motion

We saw above that for generic orbits in time-independent gravitational fields, the total energy is conserved or constant at any point along the orbit. Depending on the symmetries of the mass distribution, orbits generically also have other conserved quantities, which are either *constants of motion* (functions of the phase-space coordinates and time $C_C(\boldsymbol{x}, \boldsymbol{v}, t)$ that are constant along an orbit) and/or *integrals of motion* (functions of the phase-space coordinates only $C_I(\boldsymbol{x}, \boldsymbol{v})$ that are constant along an orbit). The energy is an example of an integral of motion, but in a spherical, time-independent mass distribution, orbits also have other integrals of motion: The three components of the angular momentum vector $L_x, L_y, L_z$ (and therefore also the total angular momentum $L$). 

To see this, let's compute the orbit of a random set of initial conditions in the Plummer model we used above:

In [ ]:
rng = np.random.default_rng(seed=123)

rand_x0 = rng.normal(0, 5, size=3) * u.pc
rand_v0 = rng.normal(0, 20, size=3) * u.km/u.s

rand_plummer_ts, rand_plummer_xs, rand_plummer_vs = leapfrog_integrate(
    plummer_acceleration, 
    x0=rand_x0,
    v0=rand_v0,
    dt=dt,  # we defined this earlier!
    n_steps=1024,
    acc_func_args=(plummer_M, plummer_b)
)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), 
                         sharex=True, sharey=True,
                         constrained_layout=True)

# x-y
axes[0].plot(rand_plummer_xs[0], rand_plummer_xs[1])
axes[0].set_xlabel(f'$x$ [{rand_plummer_xs.unit:latex_inline}]')
axes[0].set_ylabel(f'$y$ [{rand_plummer_xs.unit:latex_inline}]')

# x-z
axes[1].plot(rand_plummer_xs[0], rand_plummer_xs[2])
axes[1].set_xlabel(f'$x$ [{rand_plummer_xs.unit:latex_inline}]')
axes[1].set_ylabel(f'$z$ [{rand_plummer_xs.unit:latex_inline}]')

for ax in axes:
    ax.set_aspect('equal')
    
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    
fig.suptitle('Random orbit in the Plummer model', fontsize=24)

We can then compute the angular momentum vector at every timestep in the integration:

In [ ]:
L = np.cross(rand_plummer_xs, rand_plummer_vs, axis=0)
L = L.to(u.km/u.s * u.pc)
L.shape

We can then plot the value of the angular momentum in each component (and the total angular momentum) as a function of timestep:

In [ ]:
plt.figure(figsize=(6, 5))

plt.plot(plummer_ts, np.linalg.norm(L, axis=0), 
         marker='', label='$L$ (total)')

names = ['x', 'y', 'z']
for i in range(3):
    plt.plot(plummer_ts, L[i], marker='', label=f'$L_{names[i]}$')
    
plt.xlabel('timestep')
plt.ylabel('angular momentum')

plt.legend(loc='best', fontsize=14)

plt.tight_layout()

In the case of spherical, time-independent, central force fields, we saw above that energy $E$ and angular momentum $L$ (all components and the total magnitude) are integrals of motion. It turns out that the number of independent integrals of motion that an orbit has sets the degrees of freedom of an orbit in 6D phase space. That is, the reason that orbits do not fill six-dimensional volumes is because they often have many integrals of motion that reduce the dimensionality of the orbit (i.e. there are deep connections between coordinate symmetries, integrals of motion, and the geometry of orbits). However, we can create arbitrarily-many integrals of motion by considering any functions of these integrals of motion $f(E, L)$, so what is special about energy and angular momentum? Integrals of motion that reduce the number of degrees of freedom that an orbit has are called *isolating integrals of motion*: These are special integrals of motion that are "independent" (in detail, any such integrals are "in involution" such that the Poisson bracket of any pair of isolating integrals of motion must be zero; That is TMI for our purposes!). The number of isolating integrals of motion have geometric importance: Orbits exist in 6D phase space. If energy is conserved, this restricts orbits to a 5D manifold embedded in 6D phase space. In the case of spherical gravitational fields, the three components are three additional isolating integrals, which restrict the degrees of freedom to a 2D manifold: This is why generic orbits in spherical models (e.g., the Plummer model) trace out rosette patterns bound to a planar surface!  

### Exercise: Why do orbits in the point mass potential not fill a 2D space, but instead trace out a 1D curve? 

I.e. can you guess how many isolating integrals of motion there are for orbits in a point mass potential?

*Answer*: 

5! For orbits to be restricted to a line (a 1D curve, and not a 2D plane), there must be one additional isolating integral of motion relative to generic orbits in spherical mass models.

## A Quick Intro to Gala: Fast Orbit Integration

The point mass and Plummer potentials are just two examples of spherical, central mass models. A number of other spherical models appear often in Galactic dynamics contexts, most of which are just potential models resulting from spherical density distributions that have different power law exponents.

To demonstrate these other mass models, so that we do not have to implement all of the models here, we are going to switch from using our own home-brewed integration and potential definitions to using the [Gala](http://gala.adrian.pw/) Python package. Gala is maintained by me (Adrian) and was born out of my need for having fast tools for numerical orbit integration in Python. (For a quick intro to Gala, see the examples below, or the [Getting Started guide in the Gala documentation](http://gala.adrian.pw/en/latest/getting_started.html)) To start with, let's import the necessary Python subpackages:

In [ ]:
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

As an initial demonstration, let's re-compute the same random orbit we computed in the Plummer model just above: The position and velocity initial conditions are stored in `rand_x0` and `rand_v0`, and the potential parameters are available as `plummer_M` and `plummer_b`. 

The first thing we will do is define a Gala `PlummerPotential` object to represent the potential we are computing the orbit in:

In [ ]:
plummer_pot = gp.PlummerPotential(
    m=plummer_M, 
    b=plummer_b,
    units=galactic
)
plummer_pot

Here, we have to specify a unit system along with the potential parameters, which was also imported from Gala: In Gala, the `galactic` unit system is defined as (kpc, Myr, solar masses, radians):

In [ ]:
galactic

This potential object already knows how to compute, e.g., the energy, acceleration, or density for the Plummer mass model. For example, to compute the potential energy at some random position:

In [ ]:
some_position = [5.5, -.14, 3.4] * u.pc
plummer_pot.energy(some_position)

or the acceleration and density:

In [ ]:
print(plummer_pot.acceleration(some_position))
print(plummer_pot.density(some_position))

To compute an orbit with this potential object, we now have to define a new object to represent the initial conditions. We have the values we will need saved in the `rand_x0` and `rand_v0` variables, so we just need to pass these in to the `PhaseSpacePosition` class to create a Gala object to contain the initial condition data:

In [ ]:
w0 = gd.PhaseSpacePosition(
    pos=rand_x0,
    vel=rand_v0
)

To compute the orbit, we can use the `.integrate_orbit()` method on any Gala potential object:

In [ ]:
dt.to(u.Myr)  # defined up above!

In [ ]:
gala_plummer_orbit = plummer_pot.integrate_orbit(
    w0, dt=dt, n_steps=1_000)

The object returned from this orbit integration is a Gala `Orbit` object, which contains the position, velocity, and time at each timestep, but also has a number of convenience methods that enable various things, like making a quick plot of the different projections of the orbit:

In [ ]:
_ = gala_plummer_orbit.plot(units=u.pc)

Or computing the angular momentum at every timestep:

In [ ]:
gala_L = gala_plummer_orbit.angular_momentum()
gala_L.shape

Or even other quantities like the pericenter (smallest radius an orbit reaches), apocenter (largest radius an orbit reaches), or eccentricity (defined as $e = \frac{r_{\rm apo} - r_{\rm peri}}{r_{\rm apo} + r_{\rm peri}}$) of an orbit:

In [ ]:
gala_plummer_orbit.pericenter().to(u.pc)

In [ ]:
gala_plummer_orbit.apocenter().to(u.pc)

In [ ]:
gala_plummer_orbit.eccentricity()

The default integrator used by Gala is Leapfrog integration, but you can change the integrator to, e.g., an 8th order Runge–Kutta scheme (the Dormand–Prince 853 method) by passing this integrator to `integrate_orbit()`

In [ ]:
gala_plummer_orbit_dop853 = plummer_pot.integrate_orbit(
    w0, dt=dt, n_steps=1_000, Integrator=gi.DOPRI853Integrator)

### Exercise: Compute the energy of the DOP853 orbit at all timesteps

*Hint: there is a useful method on `Orbit` objects for doing this!*

In [ ]:
plummer_dop853_E = gala_plummer_orbit_dop853.energy()

## Common Spherical Potential Models: Isochrone Potential
   
The Isochrone potential is a special mass model in which the potential is given by 

$$
\begin{align}
\Phi(r) &= - \frac{G \, M}{\sqrt{r^2 + b^2} + b}\\
&= - \frac{G \, M}{a + b}\\
a &\equiv \sqrt{r^2 + b^2}
\end{align}
$$

where $M$ is the total mass and $b$ is a scale radius parameter. The corresponding density distribution is 

$$
\rho(r) = M \, \left( \frac{3\,(a+b)\,a^2 - r^2\,(3a + b)}{4\pi\,(a+b)^3 \, a^3} \right) \quad .
$$

This density function arises from softening a point mass: Even at $r=0$, this model has finite density. Well within the scale radius ($r \ll b$), the density is approximately constant:

$$
\rho_{r \ll b} \approx \frac{3 M}{16\pi \, b^3}
$$

Recalling Poisson's equation, this implies that the potential in this region scales with $r^2$ (note: not $1/r^2$!): This is the potential of a uniform density (i.e. "homogeneous") sphere. The potential of a uniform density sphere is in some sense an extremum for us to consider, because most relevant mass models will have decreasing density with increasing radius.

At the other limit in the Isochrone potential, i.e. at large radius, $r \gg b$ and $a \gg b$, so to first order, the potential is:

$$
\Phi_{r\gg b} \approx - \frac{G \, M}{r}\\
$$

which you may recall is the potential of a point mass! 

The Isochrone model is therefore a special model because it can "interpolate" between the potential of a uniform density sphere at small radius, and the potential of a point mass at large radius. At intermediate radii, depending on the setting of $b$, the isochrone model can be used as a good *local* approximation to many other potential models. This is useful because it is often more efficient to compute an orbit in the Isochrone model over other potentials. 

### Example: Compute orbits in different regimes of the Isochrone potential

As a demonstration, we will compute 3 orbits in the Isochrone potential: One at small radius ($r\ll b$), one at intermediate radius ($r \sim b$), and one at large radius ($r \gg b$). We will use Gala to to the orbit integration:

In [ ]:
iso_b = 10 * u.kpc
isochrone_pot = gp.IsochronePotential(
    m=1e12*u.Msun, b=iso_b, units=galactic)

Here, we will define 3 sets of initial conditions for the three orbits and store these all in an array-valued `PhaseSpacePosition` object:

In [ ]:
iso_x0 = ([
    [0.05, 0, 0],
    [1, 0, 0],
    [1000, 0, 0]
] * iso_b).T

vcirc = isochrone_pot.circular_velocity(iso_x0)
# The line below uses a Numpy paradigm called "array broadcasting"
# to fill out the 3 x 3 array of initial velocities for the 3 orbits
# https://numpy.org/doc/stable/user/basics.broadcasting.html
iso_v0 = (vcirc[np.newaxis] * np.array([0.5, 1, 0])[:, np.newaxis])

iso_w0 = gd.PhaseSpacePosition(pos=iso_x0, vel=iso_v0)
iso_w0.shape

We could then use `Potential.integrate_orbit()` to compute the three orbits simultaneously. However, here we expect the orbits to have very different orbital periods. We can get a crude estimate of the expected orbital period assuming circular orbits:

In [ ]:
iso_periods = (2*np.pi * iso_w0.physicsspherical.r / vcirc).decompose(galactic)
iso_periods

We can see that the expected orbital periods span about 2 orders of magnitude, so we would waste a lot of computational time if we used the same (small) timestep for all three orbits. Instead, we will use a fraction of the period as our timestep, and we will compute the orbits separately:

In [ ]:
iso_orbits = []
for i in range(3):
    dt = iso_periods[i] / 256  # 256 timesteps per period
    iso_orbit = isochrone_pot.integrate_orbit(iso_w0[i], dt=dt, n_steps=4096)
    iso_orbits.append(iso_orbit)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, orbit in zip(axes, iso_orbits):
    orbit.plot(['x', 'y'], axes=[ax])
    lim = 1.2 * orbit.apocenter().value
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    
axes[0].set_title(r'$r \ll b$')
axes[1].set_title(r'$r \sim b$')
axes[2].set_title(r'$r \gg b$')

fig.tight_layout()

Note that, as we expected from the limits discussed above, the three regimes of the Isochrone potential yield three very different orbital geometries. In the inner, uniform density sphere region (left panel), the orbit is an ellipse with the center of the ellipse at the origin. In the intermediate regime (middle panel), the orbit forms a rosette pattern. In the outer, point-mass region (right panel), the orbit is close to an ellipse with the origin at one focus (like a Keplerian orbit!). Why does the orbit shape change so much in different parts of the Isochrone potential?

*Aside:* The ability of the Isochrone potential to locally approximate other spherical mass models is also useful because the Isochrone potential is one of the only potential models in which one can analytically perform a coordinate transformation to a set of coordinates known as *action-angle coordinates*. We will not cover action-angle coordinates in much detail here, but they may come up in your research projects or in other contexts during the summer school. Briefly, action-angle coordinates — often represented as $(\boldsymbol{J}, \boldsymbol{\theta}$ — are a special set of canonical coordinates in which the dynamics are simple: The momentum coordinates are the actions $\boldsymbol{J}$, and these are isolating integrals of motion (i.e. they are constant!), and the position coordinates are the angles $\boldsymbol{\theta}$, which increase linearly with time. 

## Orbital Frequencies

What controls whether an orbit closes on itself and fills a 1D curve instead of filling a 2D, 3D, etc. volume? One way to understand this aspect of orbital geometry is to consider its *orbital frequencies*. For a generic planar orbit, as we are considering here in spherical potentials, orbits have two fundamental orbital frequencies: One is related to the frequency with which the orbit circulates around (the azimuthal frequency $\Omega_\phi$), and the other is related to the frequency with which the orbit oscillates in and out in radius (the radial frequency $\Omega_r$). In general spherical potentials, these frequencies will typically be irrational numbers, and they are set by the local slope of the density profile near an orbit. However, in certain mass models, or certain locations in general spherical models, these two frequencies can be *resonant*: Here, two frequencies are resonant if we can find two integers $n$ and $m$ such that $n\,\Omega_\phi + m\,\Omega_r = 0$. In other words, the ratio of the orbital frequencies is a ratio of integers: $-\frac{\Omega_\phi}{\Omega_r} = \frac{m}{n}$. Orbits that satisfy a resonance relation (a particular setting of $n$ and $m$) will close on themselves: For example, if $n=2$ and $m=1$, every revolution around in $\phi$, there will be exactly 2 oscillations in $r$, bringing the orbital trajectory back to exactly the same place in phase space. Note that if the ratio was irrational, the orbit would not close because it would take an arbitrarily long time to come back to exactly the same place.

Let's estimate the orbital frequencies of the three orbits above and compute the frequency ratios to see if this gives us a better understanding of why the orbits have different morphologies. A quick and hacky way of estimating the frequencies is to find all of the local maxima in the radius $r$ and azimuth $\phi$, difference the times, and take the mean. Thankfully, `scipy` provides utilities for identifying local maxima and minima, so the procedure to estimate the orbital frequencies (periods) will be: 
1. convert the orbits to spherical coordinates, 
2. use the `scipy.signal.argrelmax` function to find the indices of all local maxima of $r$ and $\phi$, 
3. use the returned indices to get the times corresponding to the maxima,
4. difference the subsequent times and take the mean of differences to estimate the period

In [ ]:
orbit_titles = [r'$r \ll b$', r'$r \sim b$', r'$r \gg b$']
for orbit, name in zip(iso_orbits, orbit_titles):
    # 1. Convert to spherical coordinates:
    sph = orbit.physicsspherical
    
    # 2. Compute the indices of local maxima for r and phi:
    r_idx, = argrelmax(sph.r.value)
    phi_idx, = argrelmax(sph.phi.value)
    
    # 3. Retrieve the corresponding times:
    r_times = orbit.t[r_idx]
    phi_times = orbit.t[phi_idx]
    
    # 4. Difference the times and take the mean to estimate 
    # the orbital periods in each coordinate:
    P_r = np.mean(np.diff(r_times))
    P_phi = np.mean(np.diff(phi_times))
    
    # Convert to frequencies:
    Omega_r = 2*np.pi*u.rad / P_r
    Omega_phi = 2*np.pi*u.rad / P_phi
    
    # Print the frequency ratio:
    display(Math(f'{name}  :  $\Omega_\phi / \Omega_r$ = {Omega_phi / Omega_r:.3f}'))

For the inner (~uniform density; left) orbit, the frequency ratio is very close to 1/2, meaning that for every revolution in $\phi$, there is very close to 2 oscillations in $r$! This explains the fact that the orbital shape looks like an ellipse, but with the center of the ellipse at the origin.

For the intermediate orbit (middle), the frequency ratio looks irrational, implying that the orbit should fill a 2D region. This explains the rosette pattern we see in the middle panel of the figure above. The fact that this frequency ratio is between 0.5 and 1 means that the radial frequency is always longer (i.e. the period is shorter), so orbits tend to complete between 1 and 2 radial oscillations for every azimuthal revolution.

For the outer orbit (~point mass; right), the frequency ratio is very close to 1, meaning that for every revolution in $\phi$, there is very close to 1 oscillation in $r$. This explains the elliptical shape and property of Keplerian orbits: The focus of the ellipse is at the origin because when the orbit reaches phase = 1/2 in azimuth, it also reaches phase = 1/2 in radius.

Orbits in most common spherical models relevant for Galactic dynamics have irrational azimuthal-to-radial frequency ratios between 0.5 and 1, meaning that generic orbits typically look like rosettes.

## Common Spherical Potential Models: Power-law Models

Many other commonly-used spherical potentials are derived from either "single power" or "two power" density distributions. 

### Single-power Models

Single-power density distributions have a form like:

$$
\rho(r) = \frac{\rho_0}{(r/r_0)^\alpha}
$$

Which implies a mass enclosed profile that depends on the power-law index $\alpha$:

$$
M(<r) \propto r^{3-\alpha}
$$

If you recall the expressions for the circular velocity

$$
v_\textrm{circ}(r) = \sqrt{\frac{G\,M(<r)}{r}}
$$

this implies that power-law density profiles have circular velocity curves:

$$
v^2_{\textrm{circ}, \textrm{SPL}}(r) \propto \frac{r^{3-\alpha}}{r}\\
v_{\textrm{circ}, \textrm{SPL}}(r) \propto r^{1-\alpha/2}
$$

When $\alpha = 2$, this implies that the circular velocity is independent of radius! This case corresponds to a potential that has the form:

$$
\Phi_{\alpha=2}(r) = \frac{1}{2}v_c^2 \, \ln\left(\frac{r}{r_0}\right)
$$

This *Logarithmic potential* is sometimes used as a simple model for galaxies, because they are observed to have approximately flat rotation curves (implying a close-to-constant circular velocity). 

#### Example: Comparing the circular velocity curves of several potential models

Here, we will use Gala to compute and plot the circular velocity curves of a few of the potential models we have introduced: point mass, Plummer, Isochrone, and Logarithmic

In [ ]:
potentials = {
    'Point mass': gp.KeplerPotential(m=1),
    'Plummer': gp.PlummerPotential(m=1, b=1),
    'Isochrone': gp.IsochronePotential(m=1, b=1),
    'Logarithmic': gp.LogarithmicPotential(v_c=1, r_h=1)
}

r_grid = np.linspace(0, 10, 1024)
xyz_grid = np.zeros((3, len(r_grid)))
xyz_grid[0] = r_grid

fig, ax = plt.subplots(figsize=(6, 4))
for name, pot in potentials.items():
    ax.plot(r_grid, pot.circular_velocity(xyz_grid),
            marker='', lw=2, label=name)
    
ax.set_xlim(0, 10)
ax.set_ylim(0, 2)

ax.legend(loc='upper right', fontsize=12)

ax.set_xlabel('radius')
ax.set_ylabel('circular velocity')

### Two-power Models

Two-power density distributions have a form like:

$$
\rho(r) = \frac{\rho_0}{(r/r_0)^\alpha \, (1 + r/r_0)^{\beta - \alpha}}
$$

Many useful potential models are found from setting particular values of $\beta$ and $\alpha$:

* NFW model (Navarro–Frenk–White): $(\alpha, \beta) = (1, 3)$
* Hernquist model: $(\alpha, \beta) = (1, 4)$
* Jaffe model: $(\alpha, \beta) = (2, 4)$
* Dehnen models: $\beta = 4$

Let's compare the potential, density, mass enclosed, and circular velocity curves of all of the extended mass models we have introduced so far:

In [ ]:
potentials = {
    'Plummer': gp.PlummerPotential(m=1, b=1),
    'Isochrone': gp.IsochronePotential(m=1, b=1),
    'Logarithmic': gp.LogarithmicPotential(v_c=1, r_h=1),
    'NFW': gp.NFWPotential(m=1, r_s=1),
    'Hernquist': gp.HernquistPotential(m=1, c=1),
    'Jaffe': gp.JaffePotential(m=1, c=1)
}

r_grid = np.linspace(0, 10, 1024)
xyz_grid = np.zeros((3, len(r_grid)))
xyz_grid[0] = r_grid

for func_name, ylim in zip(
    ['energy', 'density', 'mass_enclosed', 'circular_velocity'],
    [(-2, 2), (1e-8, 100), (-0.5, 3), (-0.1, 1.1)]
):
    fig, ax = plt.subplots(figsize=(6, 4))
    for name, pot in potentials.items():
        ax.plot(r_grid, getattr(pot, func_name)(xyz_grid),
                marker='', lw=2, label=name)

    ax.set_xlim(0, 10)
    ax.set_ylim(ylim)
    
    if func_name == 'density':
        ax.set_yscale('log')

    ax.legend(loc='best', fontsize=10)

    ax.set_xlabel('radius')
    ax.set_ylabel(func_name)

All of the spherical mass models above have utility in different contexts: Some have density profiles that are similar to observed light distributions in galaxies, some have circular velocity curves that match observed rotation curves in galaxies, and some are just fast to compute! There are many other possible choices for mass models that have, for example, truncated density distributions, or more complex radial dependencies, but often at the expense of simplicity: For example, some will require numerically solving for the potential or gradient, which introduces some overhead over the (mostly) analytic models discussed above. You may therefore encounter cases where people use sums or combinations of the models given above as a way of approximating more complex mass distributions, because potentials are additive.

So far, we have only discussed spherical mass distributions and the properties of orbits in spherical models. In the next tutorial notebook, we will introduce some common axisymmetric and triaxial density distributions and we will explore what happens to orbits when we consider more complex mass distributions.